# The Discrete Logarithm Problem

**Module 05** | 05-discrete-log-diffie-hellman

*Definition, brute force, discrete_log()*

## Objectives

By the end of this notebook you will be able to:

1. Understand the core ideas behind **the discrete logarithm problem**.
2. Explore these concepts interactively using SageMath.
3. Build intuition through hands-on computation and visualization.

## Prerequisites

- Basic familiarity with Python syntax.
- A working SageMath installation (or access to CoCalc/SageMathCell).

## The DLP Definition

In [ ]:
# Given g, h in a group, find x such that g^x = h
p = 23
g = Mod(5, p)
x_secret = 13
h = g^x_secret
print(f'g={g}, h={h}, find x such that g^x = h')

## Brute Force Search

In [ ]:
# Try all possible exponents (only feasible for small groups)
p = 23
g = Mod(5, p)
h = Mod(g^13, p)
for x in range(p):
    if g^x == h:
        print(f'Found: x = {x}')
        break

## Using discrete_log()

In [ ]:
# SageMath's built-in DLP solver
p = next_prime(10^6)
g = Mod(primitive_root(p), p)
x_secret = randint(2, p-2)
h = g^x_secret
x_found = discrete_log(h, g)
print(f'Secret: {x_secret}, Found: {x_found}')

## Exercises

Try these on your own before moving on:

1. **Exercise 1:** *(TODO: add exercise)*
2. **Exercise 2:** *(TODO: add exercise)*
3. **Exercise 3:** *(TODO: add exercise)*

## Summary

In this notebook we explored **the discrete logarithm problem**.  Key takeaways:

- *(TODO: summarize key point 1)*
- *(TODO: summarize key point 2)*
- *(TODO: summarize key point 3)*

**Next:** [Primitive Roots and Generators](05b-primitive-roots-generators.ipynb)